In [1]:
from ipfml import processing
from ipfml import utils
from ipfml import metrics
from PIL import Image
from scipy import signal
from skimage import color
import scipy.stats as stats
import seaborn as sns
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

from pylab import *
from skimage import data, io, color
from skimage import feature

import cv2

In [4]:
data_folder = "../fichiersSVD_light"

# Local Binary Pattern analysis on Synthesis Images for noise dectection 

## Utils functions definition

In [5]:
def compute_images_path(dict_data):
    
    all_images_path = []
    for cur_dict in dict_data:
        scene = cur_dict['name']
        prefix = cur_dict['prefix']
        indices = cur_dict['indices']

        scene_images_path = []
        for index in indices:
            path = os.path.join(data_folder, os.path.join(scene, prefix + index + ".png"))
            scene_images_path.append(path)
            
        all_images_path.append(scene_images_path)
            
    return all_images_path

In [6]:
def display_sv_data(dict_data, interval, all_images_path):
    
    sv_values = []
    plt.figure(figsize=(25, 20))
    begin, end = interval
    
    for id_dict, cur_dict in enumerate(dict_data):
        
        scene_name = cur_dict['name']
        image_indices = cur_dict['indices']
        scene_sv_values = []
    
        for id_img, img_path in enumerate(all_images_path[id_dict]):
            img = Image.open(img_path)
            print(img_path)
            
            blocks = processing.divide_in_blocks(img, (200, 200))
            block = np.array(blocks[0])
            
            if block.ndim == 3:
                U, s, V = processing.get_LAB_L_SVD(block)
            else:
                U, s, V = metrics.get_SVD(block)
                
            data = s[begin:end]
            plt.plot(data, label=scene_name + '_' + str(image_indices[id_img]))
            scene_sv_values.append(data)
            
        sv_values.append(scene_sv_values)

    plt.legend(fontsize=18)
    plt.show()
    return sv_values

In [18]:
def get_sv_data(cur_dict, interval, images_path_scene, norm=False):
    
    scene_name = cur_dict['name']
    image_indices = cur_dict['indices']
    zone = cur_dict['zone']
    scene_sv_values = []
    begin, end = interval
    
    plt.figure(figsize=(25, 15))
        
    for id_img, img_path in enumerate(images_path_scene):
        img = Image.open(img_path)
        
        img = processing.divide_in_blocks(img, (200, 200))[zone]
        
        # Convert to L canal
        img_grey = np.asarray(metrics.get_LAB_L(img), 'uint8')
        
        data_new = whiten(img_grey)
        data_new = np.diagonal(data_new)
        
        data = processing.get_LAB_L_SVD_s(img)
        
        #Image.fromarray(new_img).show()
        
        if norm:
            data_new = utils.normalize_arr(data_new)
            data = utils.normalize_arr(data)
        
        plt.plot(data, label=scene_name + '_' + str(image_indices[id_img] + ' (new)'))
        plt.plot(data_new, label=scene_name + '_' + str(image_indices[id_img]))
        scene_sv_values.append(data)
    
    plt.legend(fontsize=18)
    plt.show()
    return scene_sv_values


In [3]:
class LocalBinaryPatterns:
	def __init__(self, numPoints, radius):
		# store the number of points and radius
		self.numPoints = numPoints
		self.radius = radius
 
	def describe(self, image, eps=1e-7):
		# compute the Local Binary Pattern representation
		# of the image, and then use the LBP representation
		# to build the histogram of patterns
		lbp = feature.local_binary_pattern(image, self.numPoints,
			self.radius, method="uniform")
		(hist, _) = np.histogram(lbp.ravel(),
			bins=np.arange(0, self.numPoints + 3),
			range=(0, self.numPoints + 2))
 
		# normalize the histogram
		hist = hist.astype("float")
		hist /= (hist.sum() + eps)
 
		# return the histogram of Local Binary Patterns
		return hist

## Scenes zones data

In [7]:
# start 00020 - ref 00900 - step 10
dict_appart = {'name': 'Appart1opt02', 
               'prefix': 'appartAopt_', 
               'indices': ["00020", "00200", "00300", "00900"],
               'zone': 6}

# start 00050 - ref 01200 - step 10
dict_cuisine = {'name': 'Cuisine01', 
               'prefix': 'cuisine01_', 
               'indices': ["00050", "00400", "01200"],
               'zone': 6}

# start 00020 - ref 00950 - step 10
dict_sdb_c = {'name': 'SdbCentre', 
               'prefix': 'SdB2_', 
               'indices': ["00020", "00400", "00950"],
               'zone': 6}

# start 00020 - ref 00950 - step 10
dict_sdb_d = {'name': 'SdbDroite', 
               'prefix': 'SdB2_D_', 
               'indices': ["00020", "00400", "00950"],
               'zone': 6}

In [8]:
all_dicts = [dict_appart, dict_cuisine, dict_sdb_c, dict_sdb_d]
interval = (30, 200)

In [9]:
images_path = compute_images_path(all_dicts)